# Create and Run chat flows using Promptflow Python SDK

### Overview

Prompt flow is a suite of development tools designed to streamline the end-to-end development cycle of LLM-based AI applications, from ideation, prototyping, testing, evaluation to production deployment and monitoring. It makes prompt engineering much easier and enables you to build LLM apps with production quality.

In this hands-on, you will be able to:
Create flows that link fine-tuned phi3.5 endpoint(Python code) and gpt model in a executable workflow.
Debug and iterate your flows, especially tracing interaction with LLMs with ease.

#### 1. Set up Promptflow client with Credential and configuration

#### 2. Create a new chat flow by providing the flow name and description.

#### 3. Run Basic Promptflow with questions to compare models

#### 4. Run Context Added Promptflow with the outdoor questions

#### 5. Use serverless endpoint to run the Promptflow with context

[Note] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.


In [5]:
%load_ext autoreload
%autoreload 2

import os, sys
lab_prep_dir = os.getcwd().split("SLMWorkshopCN")[0] + "SLMWorkshopCN/0_lab_preparation"
sys.path.append(os.path.abspath(lab_prep_dir))

from common import check_kernel
check_kernel()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Kernel: pythonundefinedundefinedundefinedjvsc74a57bd02139c70ac98f3202d028164a545621647e07f47fd6f5d8ac55cf952bf7c15ed1


In [6]:
import json
import os
import time

# Import required libraries
from promptflow.azure import PFClient
from promptflow.entities import Run
# Import required libraries
from azure.identity import DefaultAzureCredential, EnvironmentCredential, InteractiveBrowserCredential
from dotenv import load_dotenv
from azure.core.exceptions import HttpResponseError
load_dotenv()
with open('./config_prd.json', 'r') as f:
    config = json.load(f)
    
print(config["subscription_id"])
print(config["resource_group"])
print(config["workspace_name"]) # Azure AI Studio project name which is not the same as the Azure ML workspace name

49aee8bf-3f02-464f-a0ba-e3467e7d85e2
es1u1aif1grp
esu1aif1prj


In [7]:
from tqdm import tqdm

# Monitor the status of the run_result
def monitor_status(pf_azure_client:PFClient, run_result:Run):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = pf_azure_client.runs.get(run_result).status
        if status == "Preparing":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = pf_azure_client.runs.get(run_result).status
        pbar.update(1)
        print("Promptflow Running Completed")

## 1. Set up Promptflow client with Credential and configuration

-   Create a promptflow client with the credential and configuration. You need to set the `config_prd.json` file with subscription_id, resource_group and workspace_name


In [8]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
# if you cannot use DefaultAzureCredential and InteractiveBrowserCredential you need to set up the Managed identity in your .env file

pf_azure_client = PFClient.from_config(credential=credential, path="./config_prd.json")

# pf_azure_client = PFClient(credential=credential, 
#                            subscription_id="your subscription id", 
#                            resource_group_name="your resource group name", 
#                            workspace_name="your workspace name")            

try:
    workspace = pf_azure_client.ml_client.workspaces.get(name=config["workspace_name"])
    print(f"Connected to Azure AI Studio Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")


Found the config file in: config_prd.json


Connected to Azure AI Studio Workspace: esu1aif1prj
Workspace Location: eastus
Workspace ID: /subscriptions/49aee8bf-3f02-464f-a0ba-e3467e7d85e2/resourceGroups/es1u1aif1grp/providers/Microsoft.MachineLearningServices/workspaces/esu1aif1prj


## 2. Create a new chat flow by providing the flow name and description.

-   Create a new chat flow by providing the flow name and description. You can view and clone the flow on Azure AI studio UI.

> ✨ **_important_** <br>
> Grant the Storage File Data Privileged Contributor, Storage Blob Data Contributor at the storage of AI studiorole to user, group, service principle and managed Identity which you are trying to access, you can execute the code below.


### Check the exist connections

-   currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
    > ✨ **_important_** <br>
    > Update flow.dag.yaml files in your flow_path with the connection name you have created in the Azure ML Studio UI.


In [9]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

env = Environment(loader=FileSystemLoader('.'))

# Read the template file
template = env.get_template('./flow-template/chat.flow.dag.yaml')

# Define the variables for the template
variables = {
	"your_phi35_connection_name": "coneu1-phi3-endpoint-241201",
	"your_gpt4o_connection_name": "aoai-tst1",
}

rendered_content = template.render(variables)
Path('./chat/flow.dag.yaml').write_text(rendered_content)

print(Path('./chat/flow.dag.yaml').read_text()) 

$schema: https://azuremlschemas.azureedge.net/promptflow/latest/Flow.schema.json
environment:
  python_requirements_txt: requirements.txt
inputs:
  question:
    type: string
    is_chat_input: true
    default: What is the capital of France?
outputs:
  phi35_answer:
    type: string
    reference: ${phi35.output}
    is_chat_output: false
  gpt4o_answer:
    type: string
    reference: ${gpt4o.output}
    is_chat_output: true
nodes:
- name: phi35
  type: python
  source:
    type: code
    path: phi35_finetuned.py
  inputs:
    connection: coneu1-phi3-endpoint-241201
    input_data: ${inputs.question}
- name: gpt4o
  type: llm
  source:
    type: code
    path: chat.jinja2
  inputs:
    deployment_name: gpt-4o
    temperature: 0.7
    top_p: 1
    max_tokens: 256
    question: ${inputs.question}
  connection: aoai-tst1
  api: chat
  module: promptflow.tools.aoai
  use_variants: false


In [ ]:
#Grant the right accessibility to create the flow 
pf_azure_client.flows.create_or_update(flow="chat/", type="chat", display_name="comparison flow created from python sdk", description="fine-tuned model comparison flow")#在AI Foundry/AI Project/Prompt Flow/Flow标签页中可看到

Flow created successfully:
{
    "name": "93a899e9-7106-4633-9329-3eef4ecc9898",
    "type": "chat",
    "description": "fine-tuned model comparison flow",
    "path": "Users/luogang/promptflow/chat-12-15-2024-08-19-36/flow.dag.yaml",
    "code": "azureml://locations/eastus/workspaces/6f6dfe14-cbed-4c2f-8e06-39f5bfbc7aaf/flows/93a899e9-7106-4633-9329-3eef4ecc9898",
    "display_name": "comparison flow created from python sdk",
    "owner": {
        "user_object_id": "a0490c2e-4f6d-490a-9f03-0b49b8a3c480",
        "user_tenant_id": "16b3c013-d300-468d-ac64-7eda0820b6d3",
        "user_name": "Gang Luo"
    },
    "is_archived": false,
    "created_date": "2024-12-15 08:19:44.673901+00:00",
    "flow_portal_url": "https://ai.azure.com/projectflows/93a899e9-7106-4633-9329-3eef4ecc9898/6f6dfe14-cbed-4c2f-8e06-39f5bfbc7aaf/details/Flow?wsid=/subscriptions/49aee8bf-3f02-464f-a0ba-e3467e7d85e2/resourcegroups/es1u1aif1grp/providers/Microsoft.MachineLearningServices/workspaces/esu1aif1prj"
}


## 3. Run Basic Promptflow with questions to compare models

-   Run the Promptflow with the simple questions such as "What is the capital of France?" and compare the results of the models


In [ ]:
flow_path = "./chat"
data_path = "./data/questions_basic.jsonl"

column_mapping = {
    "question": "${data.question}"
}

run_result = pf_azure_client.run(#在AI Foundry/AI Project/Prompt Flow/Runs标签页中可看到，这里代码提交的run与上面pf_azure_client.flows.create_or_update创建的flow并无关联，只是丢个xml上去跑而已
    flow=flow_path,
    type="chat",
    data=data_path,
    column_mapping=column_mapping,
    display_name="chat_with_data",
    tags={"chat_with_jsonl": "", "1st_round": ""},
)


Uploading questions_basic.jsonl (< 1 MB): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 724/724 [00:00<00:00, 7.96kB/s]


Portal url: https://ai.azure.com/projectflows/trace/run/chat_variant_0_20241215_081945_455389/details?wsid=/subscriptions/49aee8bf-3f02-464f-a0ba-e3467e7d85e2/resourcegroups/es1u1aif1grp/providers/Microsoft.MachineLearningServices/workspaces/esu1aif1prj


In [12]:
monitor_status(pf_azure_client, run_result)

Running Status:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/3 [00:00<?, ?step/s]

Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: Preparing


Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:45<01:31, 45.63s/step]

Current Status: Running


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [00:56<00:25, 25.36s/step]

Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:07<00:00, 22.65s/step]

Promptflow Running Completed


In [13]:
detail = pf_azure_client.get_details(run_result)

detail

,inputs.question,inputs.line_number,outputs.phi35_answer,outputs.gpt4o_answer
outputs.line_number,,,,
0,what is the center of Seoul?,0,The center of Seoul is often considered to be...,The center of Seoul is often considered to be ...
1,What is the capital of France?,1,The capital of France is Paris.,The capital of France is Paris.
2,Tell me about the brief history of Microsoft,2,Microsoft was founded by Bill Gates and Paul ...,Microsoft was founded by Bill Gates and Paul A...
3,Who wrote Romeo and Juliet?,3,"William Shakespeare wrote ""Romeo and Juliet.""","William Shakespeare wrote ""Romeo and Juliet."""
4,What does HTML stand for?,4,HTML stands for HyperText Markup Language. It...,HTML stands for HyperText Markup Language.
5,Who painted the Mona Lisa?,5,The Mona Lisa was painted by Leonardo da Vinci.,Leonardo da Vinci painted the Mona Lisa.
6,What is the largest planet in our solar system?,6,Jupiter is the largest planet in our solar sy...,The largest planet in our solar system is Jupi...
7,Who won the Nobel Prize in Literature in 2024?,7,"I'm sorry, but as of my knowledge cutoff in e...","I'm sorry, but I don't have information about ..."


## 4. Run Context Added Promptflow with the outdoor questions

-   Run the Promptflow using the context data and ask the outdoor product related questions to compare the results of the models


### Check the exist connections

-   currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
    > ✨ **_important_** <br>
    > Update flow.dag.yaml files in your flow_path with the connection name you have created in the Azure ML Studio UI.


In [14]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

env = Environment(loader=FileSystemLoader('.'))

# Read the template file
template = env.get_template('./flow-template/chat-context.flow.dag.yaml')

# Define the variables for the template
variables = {
	"your_phi35_connection_name": "coneu1-phi3-endpoint-241201",
	"your_gpt4o_connection_name": "aoai-tst1"
}

rendered_content = template.render(variables)
Path('./chat-context/flow.dag.yaml').write_text(rendered_content)

print(Path('./chat-context/flow.dag.yaml').read_text()) 

$schema: https://azuremlschemas.azureedge.net/promptflow/latest/Flow.schema.json
environment:
  python_requirements_txt: requirements.txt
inputs:
  question:
    type: string
    is_chat_input: true
    default: What is the capital of France?
  context:
    type: string
    is_chat_input: false
    default: TrailMaster X4 Tent is a durable polyester tent
outputs:
  phi35_answer:
    type: string
    reference: ${phi35.output}
    is_chat_output: false
  gpt4o_answer:
    type: string
    reference: ${gpt4o.output}
    is_chat_output: true
nodes:
- name: phi35
  type: python
  source:
    type: code
    path: phi35_finetuned.py
  inputs:
    connection: coneu1-phi3-endpoint-241201
    question: ${inputs.question}
    context: ${inputs.context}
- name: gpt4o
  type: llm
  source:
    type: code
    path: chat.jinja2
  inputs:
    deployment_name: gpt-4o
    temperature: 0.7
    top_p: 1
    max_tokens: 256
    question: ${inputs.question}
  connection: aoai-tst1
  api: chat
  module: pro

In [15]:
flow_path = "./chat-context"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    #context_simple.json中只有固定的一条ctx所以可以这样写
}

run_result_with_context = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_context_data",
    tags={"chat_with_context_jsonl": "", "1st_round": ""},
)


Uploading questions_outdoor.jsonl (< 1 MB): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467/467 [00:00<00:00, 5.70kB/s]


Portal url: https://ai.azure.com/projectflows/trace/run/chat_context_variant_0_20241215_082112_494255/details?wsid=/subscriptions/49aee8bf-3f02-464f-a0ba-e3467e7d85e2/resourcegroups/es1u1aif1grp/providers/Microsoft.MachineLearningServices/workspaces/esu1aif1prj


In [16]:
monitor_status(pf_azure_client, run_result_with_context)

Running Status:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/3 [00:00<?, ?step/s]

Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: Preparing


Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:56<01:52, 56.17s/step]

Current Status: Running


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [01:07<00:29, 29.65s/step]

Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:18<00:00, 26.19s/step]

Promptflow Running Completed


In [17]:
detail = pf_azure_client.get_details(run_result_with_context)

detail

,inputs.question,inputs.context,inputs.line_number,outputs.phi35_answer,outputs.gpt4o_answer
outputs.line_number,,,,,
0,tell me about your TrailMaster X4 Tent,TrailMaster X4 Tent is a durable polyester ten...,0,The TrailMaster X4 Tent is a four-person poly...,"The TrailMaster X4 Tent is a spacious, durable..."
1,Do you have any climbing gear?,TrailMaster X4 Tent is a durable polyester ten...,1,"No, the provided context does not include any...","I'm just a digital assistant, so I don't have ..."
2,Can you tell me about your selection of tents?,TrailMaster X4 Tent is a durable polyester ten...,2,"The TrailMaster X4 Tent is a four-person, dur...","I wish I could sell you a tent, but as an AI a..."
3,Do you have TrekReady Hiking Boots? How much i...,TrailMaster X4 Tent is a durable polyester ten...,3,I'm an AI and don't have access to current pr...,I can help you with that! While I don't have t...


## 5. Use serverless endpoint to run the Promptflow with context

-   Create a serverless endpoint to run the Promptflow with the context. You can use the endpoint to run the flow with the context.


### deploy your serverless endpoint

-   go to the Azure AI studio > Model catalog > search phi-3.5 > deply Phi-3.5-mini-instruct as your serverless endpint
    <br>
    ![serverless endpoint](./images/deploy_serverless_endpoint.jpg)
    <br>
    <br>
-   once the deployment is done, go to Deployments and you can see the endpoint deployed in the endpoint section. Click to check the details and copy key and phi35-mini-instruct: Chat Completion endpoint url
    ![copy connection](./images/copy_connection.jpg)
    <br>
    <br>
-   go to Settings in Azure AI studio > Connections > create a new connection naming phi35-serverless with the copied key and endpoint url
    ![create new serverless connection](./images/create_new_serverless_connection.jpg)


### Check the exist connections

-   currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
    > ✨ **_important_** <br>
    > Update flow.dag.yaml files in your flow_path with the connection name you have created in the Azure ML Studio UI.


In [18]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

env = Environment(loader=FileSystemLoader('.'))

# Read the template file
template = env.get_template('./flow-template/chat-serverless.flow.dag.yaml')

# Define the variables for the template with your connection names for chat serverless 
variables = {
	"your_phi35_serverless_connection_name": "conphi35mini-istru-esu1aif1dpl1",
	"your_gpt4o_connection_name": "aoai-tst1"
}

rendered_content = template.render(variables)
Path('./chat-serverless/flow.dag.yaml').write_text(rendered_content)

print(Path('./chat-serverless/flow.dag.yaml').read_text()) 

$schema: https://azuremlschemas.azureedge.net/promptflow/latest/Flow.schema.json
environment:
  python_requirements_txt: requirements.txt
inputs:
  question:
    type: string
    is_chat_input: true
    default: What is the capital of France?
  context:
    type: string
    is_chat_input: false
    default: TrailMaster X4 Tent is a durable polyester tent
outputs:
  phi35_answer:
    type: string
    reference: ${phi35.output}
    is_chat_output: false
  gpt4o_answer:
    type: string
    reference: ${gpt4o.output}
    is_chat_output: true
nodes:
- name: phi35
  type: python
  source:
    type: code
    path: phi35_chatcompletion.py
  inputs:
    connection: conphi35mini-istru-esu1aif1dpl1
    question: ${inputs.question}
    context: ${inputs.context}
- name: gpt4o
  type: llm
  source:
    type: code
    path: chat.jinja2
  inputs:
    deployment_name: gpt-4o
    temperature: 0.2
    top_p: 1
    max_tokens: 512
    question: ${inputs.question}
  connection: aoai-tst1
  api: chat
  mo

In [19]:
flow_path = "./chat-serverless"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_serverless_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_serverless_context_data",
    tags={"chat_serverless_context_jsonl": "", "1st_round": ""},
)


Uploading chat-serverless (0.0 MBs): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4138/4138 [00:00<00:00, 12642.29it/s]


Portal url: https://ai.azure.com/projectflows/trace/run/chat_serverless_variant_0_20241215_082243_117922/details?wsid=/subscriptions/49aee8bf-3f02-464f-a0ba-e3467e7d85e2/resourcegroups/es1u1aif1grp/providers/Microsoft.MachineLearningServices/workspaces/esu1aif1prj


In [20]:
monitor_status(pf_azure_client, run_serverless_result)

Running Status:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/3 [00:00<?, ?step/s]

Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: Preparing


Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:55<01:51, 55.83s/step]

Current Status: Running


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [01:07<00:29, 29.73s/step]

Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:51<00:00, 37.20s/step]

Promptflow Running Completed


In [21]:
detail = pf_azure_client.get_details(run_serverless_result)

detail

,inputs.question,inputs.context,inputs.line_number,outputs.phi35_answer,outputs.gpt4o_answer
outputs.line_number,,,,,
0,tell me about your TrailMaster X4 Tent,TrailMaster X4 Tent is a durable polyester ten...,0,The TrailMaster X4 Tent is a robust four-pers...,The TrailMaster X4 Tent is a spacious and dura...
1,Do you have any climbing gear?,TrailMaster X4 Tent is a durable polyester ten...,1,"I'm sorry, but I don't have specific climbing...","I don't have any climbing gear myself, but I c..."
2,Can you tell me about your selection of tents?,TrailMaster X4 Tent is a durable polyester ten...,2,Certainly! The TrailMaster X4 Tent is a robus...,"I don't sell tents, but I can help you find th..."
3,Do you have TrekReady Hiking Boots? How much i...,TrailMaster X4 Tent is a durable polyester ten...,3,"I'm sorry, but I don't have real-time access ...","I don't have TrekReady Hiking Boots myself, bu..."
